# 1) Setup
---




## a. Libraries and Imports

---




In [ ]:
# !pip install trackio
# import trackio

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import kagglehub

## b. Data Loaders 
---



In [2]:
class FaceDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, train=True):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]    
        if 'full_path' in row:
            image_name = os.path.basename(row['full_path'])
        elif 'id' in row:
            image_name = str(row['id']) + ".jpg" 
            
        image_name = os.path.basename(row['full_path']) if 'full_path' in row else str(row['id'])
            
        image_path = os.path.join(self.img_dir, image_name)
        
        try:
            image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            image = Image.new("RGB", (224, 224), (128, 128, 128)) 

        if self.transform:
            image = self.transform(image)

        if self.train:
            age = torch.tensor(row["age"], dtype=torch.float32)
            gender = torch.tensor(row["gender"], dtype=torch.long)
            return image, age, gender
        else:
            return image, row["id"]

In [ ]:
# #    ScratchCNN tranforms & data loaders



# transform = transforms.Compose([
#     transforms.Resize((128, 128)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

# train_img_dir = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train"
# df = pd.read_csv("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv")

# train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# train_df.to_csv("train_split.csv", index=False)
# val_df.to_csv("val_split.csv", index=False)

# train_dataset = FaceDataset("train_split.csv", train_img_dir, transform=transform, train=True)
# val_dataset   = FaceDataset("val_split.csv",   train_img_dir, transform=transform, train=True)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
# val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=2)


In [ ]:
# #  preTrained model ke liye transforms & data loaders



# transform_Pretrained = transforms.Compose([
#     transforms.Resize((224, 224)), # ResNet needs 224x224
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
#                          std=[0.229, 0.224, 0.225])
# ])

# #adding augmentations to prevent overfitting (since abhi bade pretrained models use kr rhe)
# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
#                          std=[0.229, 0.224, 0.225])
# ])


# train_img_dir = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train"
# df = pd.read_csv("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv")

# train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# train_df.to_csv("train_split.csv", index=False)
# val_df.to_csv("val_split.csv", index=False)

# train_dataset = FaceDataset("train_split.csv", train_img_dir, transform=train_transform, train=True)
# val_dataset   = FaceDataset("val_split.csv",   train_img_dir, transform=transform_Pretrained, train=True)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
# val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=2)

## c. Model Architecture





---



### c1) Scratch CNN

In [ ]:
# class ScratchCNN(nn.Module):
#     def __init__(self):
#         super(ScratchCNN, self).__init__()

#         self.features = nn.Sequential(
#             nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
#             nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
#             nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
#         )

#         self.flatten = nn.Flatten()

#         # Input image size 128x128 -> 16x16 feature map (3 max pools)
#         self.fc = nn.Sequential(
#             nn.Linear(128 * 16 * 16, 256),
#             nn.ReLU(),
#         )

#         self.age_head = nn.Linear(256, 1)        # regression
#         self.gender_head = nn.Linear(256, 2)     # classification

#     def forward(self, x):
#         x = self.features(x)
#         x = self.flatten(x)
#         x = self.fc(x)

#         age = self.age_head(x)
#         gender = self.gender_head(x)
#         return age, gender

In [ ]:
# #Deeper ScratchCNNv2 with Batch Norm and Dropout
# class ScratchCNNv2(nn.Module):  
#     def __init__(self):
#         super(ScratchCNNv2, self).__init__()

#         self.features = nn.Sequential(
#             # Block 1: 128 -> 64
#             nn.Conv2d(3, 32, kernel_size=3, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             # Block 2: 64 -> 32
#             nn.Conv2d(32, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             # Block 3: 32 -> 16
#             nn.Conv2d(64, 128, kernel_size=3, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             # Block 4: 16 -> 8
#             nn.Conv2d(128, 256, kernel_size=3, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )

#         self.flatten = nn.Flatten()

#         # Final feature map size is 8x8 (128 -> 64 -> 32 -> 16 -> 8)
#         # Final channels = 256
#         self.fc = nn.Sequential(
#             nn.Linear(256 * 8 * 8, 512),
#             nn.ReLU(),
#             nn.Dropout(0.5), 
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Dropout(0.3)
#         )

#         # Output heads
#         self.age_head = nn.Linear(256, 1)      # Regression
#         self.gender_head = nn.Linear(256, 2)   # Classification

#     def forward(self, x):
#         x = self.features(x)
#         x = self.flatten(x)
#         x = self.fc(x)
#         age = self.age_head(x)
#         gender = self.gender_head(x)
#         return age, gender


### c2) Finetuned CNN

In [3]:
from torchvision import models

class ResNet34MultiTask(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet34MultiTask, self).__init__()
        
        weights = models.ResNet34_Weights.DEFAULT if pretrained else None
        self.backbone = models.resnet34(weights=weights)

        num_features = self.backbone.fc.in_features  # num. of features in final layer
        
        self.backbone.fc = nn.Identity() #final fc layer ko -> Identity layer, so num_fea

        # new multi-task heads
        self.age_head = nn.Linear(num_features, 1)      # Regression to 1 value
        self.gender_head = nn.Linear(num_features, 2)   # Classification to 2 values

    def forward(self, x):
        features = self.backbone(x) #features from backbone
        
        # Pass features to each head
        age = self.age_head(features)
        gender = self.gender_head(features)
        
        return age, gender

## d. Loss Function, Optimizer, and Metrics
---



In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# criterion_age = nn.MSELoss()
# criterion_gender = nn.CrossEntropyLoss()
# GENDER_LOSS_WEIGHT = 50.0 

In [ ]:
# # == ScratchCNNv2
# model = ScratchCNNv2().to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) 

In [ ]:
# # == Resnet34

# model = ResNet34MultiTask(pretrained=True).to(device)

# # Optimizers for 2-Phase Training 

# # PHASE 1: Optimizer for the head ONLY
# head_params = list(model.age_head.parameters()) + list(model.gender_head.parameters())
# optimizer_head = torch.optim.Adam(head_params, lr=1e-4) 

# # PHASE 2: Optimizer for the ENTIRE model
# optimizer_all = torch.optim.Adam(model.parameters(), lr=1e-5) 

# 2. Training Code 

*(💡 This section should be commented while submitting to prevent re-training)*

---



## 🧭a. Trackio Initialization and Logging

---



In [ ]:
# # --- Hugging Face Login ---
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient
# try:
#     user_secrets = UserSecretsClient()
#     secret_value_0 = user_secrets.get_secret("HF_TOKEN")
#     login(token=secret_value_0)
#     print("Hugging Face login successful.")
# except Exception as e:
#     print(f"Hugging Face login failed. Error: {e}")

# # --- Trackio Initialization ---
# import trackio
# try:
#     trackio.init(
#         project="25-t3-nppe1",
#         space_id="archi29/dlgenai-nppe",  
#         group="pretrained_1_v3"
#     )
#     print("Trackio initialized for Model 3 (ResNet34).")
# except Exception as e:
#     print(f"Trackio initialization failed: {e}")

## 🧩b. Training Execution

---



In [ ]:
# print("--- Starting Model 2 (ScratchCNNv2) Training ---")

# epochs = 30 
# best_val_rmse = float("inf") 

# for epoch in range(epochs):
#     model.train()
#     total_train_loss = 0
#     total_train_age_loss = 0
#     total_train_gender_loss = 0

#     for imgs, ages, genders in train_loader:
#         imgs, ages, genders = imgs.to(device), ages.to(device), genders.to(device)

#         optimizer.zero_grad()
#         pred_age, pred_gender = model(imgs)

#         loss_age = criterion_age(pred_age.squeeze(), ages)
#         loss_gender = criterion_gender(pred_gender, genders)
        
#         # loss weighting
#         loss = loss_age + (GENDER_LOSS_WEIGHT * loss_gender)

#         loss.backward()
#         optimizer.step()

#         total_train_loss += loss.item()
#         total_train_age_loss += loss_age.item()
#         total_train_gender_loss += loss_gender.item()

#     # Validation with Competition Metrics (RMSE, F1)
#     model.eval()
#     total_val_loss = 0
    
#     # Lists to store all predictions and ground truths for metric calculation
#     all_age_preds = []
#     all_age_gts = []
#     all_gender_preds = []
#     all_gender_gts = []

#     with torch.no_grad():
#         for imgs, ages, genders in val_loader:
#             imgs, ages, genders = imgs.to(device), ages.to(device), genders.to(device)

#             pred_age, pred_gender = model(imgs)

#             loss_age = criterion_age(pred_age.squeeze(), ages)
#             loss_gender = criterion_gender(pred_gender, genders)
#             loss = loss_age + (GENDER_LOSS_WEIGHT * loss_gender)
#             total_val_loss += loss.item()

#             # Store predictions and ground truths for metrics
#             all_age_preds.append(pred_age.squeeze().cpu())
#             all_age_gts.append(ages.cpu())
#             all_gender_preds.append(pred_gender.argmax(dim=1).cpu())
#             all_gender_gts.append(genders.cpu())

#     # Calculate Metrics for the Epoch
#     avg_train_loss = total_train_loss / len(train_loader)
#     avg_val_loss = total_val_loss / len(val_loader)
    
#     # Calculate separate train losses for logging
#     avg_train_age_loss = total_train_age_loss / len(train_loader)
#     avg_train_gender_loss = total_train_gender_loss / len(train_loader)

#     # Concatenate all batches
#     age_preds_tensor = torch.cat(all_age_preds)
#     age_gts_tensor = torch.cat(all_age_gts)
#     gender_preds_tensor = torch.cat(all_gender_preds)
#     gender_gts_tensor = torch.cat(all_gender_gts)

#     # Calculate RMSE and Macro F1
#     val_rmse = np.sqrt(mean_squared_error(age_gts_tensor.numpy(), age_preds_tensor.numpy()))
#     val_macro_f1 = f1_score(gender_gts_tensor.numpy(), gender_preds_tensor.numpy(), average='macro')

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val F1: {val_macro_f1:.4f}")
#     print(f"  [Losses] Train Age: {avg_train_age_loss:.2f}, Train Gender: {avg_train_gender_loss:.2f}")

#     # --- Trackio Logging ---
#     trackio.log({
#         "epoch": epoch + 1,
#         "train_loss_total": avg_train_loss,
#         "train_loss_age": avg_train_age_loss,
#         "train_loss_gender": avg_train_gender_loss,
#         "val_loss_total": avg_val_loss,
#         "val_rmse": val_rmse,          # Log the competition metric
#         "val_macro_f1": val_macro_f1   # Log the competition metric
#     })

#     # Save Best Model 
#     if val_rmse < best_val_rmse:
#         best_val_rmse = val_rmse
#         torch.save(model.state_dict(), "cnn_scratch2.pth")
#         print(f"✅ Best model updated (RMSE: {best_val_rmse:.4f}) and saved to 'cnn_scratch2.pth'")

# print("--- Training finished ---")
# trackio.finish()

In [ ]:
# print("--- Starting Model 3 (ResNet34) 2-Phase Training ---")

# PHASE1_EPOCHS = 5     
# PHASE2_EPOCHS = 25    
# # Total epochs = 30

# # Early Stopping Parameters
# PATIENCE = 7         
# early_stop_counter = 0
# best_val_rmse = float("inf")

# total_epochs = PHASE1_EPOCHS + PHASE2_EPOCHS

# for epoch in range(total_epochs):
    
#     #2-Phase Training Logic
#     if epoch < PHASE1_EPOCHS:
#         # PHASE 1: Only train Head
#         print(f"--- Epoch {epoch+1}/{total_epochs} [Phase 1: Training Head] ---")
#         model.train() #freeze backbone
#         model.backbone.eval()   
#         optimizer = optimizer_head
    
#     elif epoch == PHASE1_EPOCHS:
#         # PHASE 2: Unfreeze All 
#         print(f"--- Epoch {epoch+1}/{total_epochs} [Phase 2: Unfreezing All Layers] ---")
#         model.train() 
#         optimizer = optimizer_all
    
#     else:
#         # PHASE 2: Fine-Tuning 
#         print(f"--- Epoch {epoch+1}/{total_epochs} [Phase 2: Fine-Tuning All] ---")
#         model.train()
#         optimizer = optimizer_all
    
    
#     total_train_loss = 0
#     total_train_age_loss = 0
#     total_train_gender_loss = 0

#     # Training Batch Loop 
#     for imgs, ages, genders in train_loader:
#         imgs, ages, genders = imgs.to(device), ages.to(device), genders.to(device)

#         optimizer.zero_grad()
#         pred_age, pred_gender = model(imgs)

#         loss_age = criterion_age(pred_age.squeeze(), ages)
#         loss_gender = criterion_gender(pred_gender, genders)
        
#         loss = loss_age + (GENDER_LOSS_WEIGHT * loss_gender)
        
#         loss.backward()
#         optimizer.step()

#         total_train_loss += loss.item()
#         total_train_age_loss += loss_age.item()
#         total_train_gender_loss += loss_gender.item()

#     #Validation with Competition Metrics (RMSE, F1)
#     model.eval() 
#     total_val_loss = 0
    
#     all_age_preds = []
#     all_age_gts = []
#     all_gender_preds = []
#     all_gender_gts = []

#     with torch.no_grad():
#         for imgs, ages, genders in val_loader:
#             imgs, ages, genders = imgs.to(device), ages.to(device), genders.to(device)
#             pred_age, pred_gender = model(imgs)

#             loss_age = criterion_age(pred_age.squeeze(), ages)
#             loss_gender = criterion_gender(pred_gender, genders)
#             loss = loss_age + (GENDER_LOSS_WEIGHT * loss_gender)
#             total_val_loss += loss.item()

#             all_age_preds.append(pred_age.squeeze().cpu())
#             all_age_gts.append(ages.cpu())
#             all_gender_preds.append(pred_gender.argmax(dim=1).cpu())
#             all_gender_gts.append(genders.cpu())

#     # Metrics for the Epoch 
#     avg_train_loss = total_train_loss / len(train_loader)
#     avg_val_loss = total_val_loss / len(val_loader)
#     avg_train_age_loss = total_train_age_loss / len(train_loader)
#     avg_train_gender_loss = total_train_gender_loss / len(train_loader)

#     age_preds_tensor = torch.cat(all_age_preds)
#     age_gts_tensor = torch.cat(all_age_gts)
#     gender_preds_tensor = torch.cat(all_gender_preds)
#     gender_gts_tensor = torch.cat(all_gender_gts)

#     val_rmse = np.sqrt(mean_squared_error(age_gts_tensor.numpy(), age_preds_tensor.numpy()))
#     val_macro_f1 = f1_score(gender_gts_tensor.numpy(), gender_preds_tensor.numpy(), average='macro')

#     print(f"Epoch {epoch+1}/{total_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val F1: {val_macro_f1:.4f}")
#     print(f"  [Losses] Train Age: {avg_train_age_loss:.2f}, Train Gender: {avg_train_gender_loss:.2f}")

#     # --- Trackio Logging ---
#     trackio.log({
#         "epoch": epoch + 1,
#         "train_loss_total": avg_train_loss,
#         "train_loss_age": avg_train_age_loss,
#         "train_loss_gender": avg_train_gender_loss,
#         "val_loss_total": avg_val_loss,
#         "val_rmse": val_rmse,
#         "val_macro_f1": val_macro_f1,
#         "phase": 1 if epoch < PHASE1_EPOCHS else 2
#     })

#     # Early Stopping & Save Best Model
#     if val_rmse < best_val_rmse:
#         best_val_rmse = val_rmse
#         torch.save(model.state_dict(), "Pretrained_1.pth")
#         print(f"✅ Best model updated (RMSE: {best_val_rmse:.4f}) and saved to 'Pretrained_1.pth'")
#         early_stop_counter = 0 
#     else:
#         early_stop_counter += 1
#         print(f"⚠️ No improvement in Val RMSE for {early_stop_counter} epoch(s). Best RMSE: {best_val_rmse:.4f}")

#     if early_stop_counter >= PATIENCE:
#         print(f"--- 🛑 Early stopping triggered after {epoch + 1} epochs. ---")
#         break 

# print("--- Training finished ---")
# trackio.finish()

## 📤c. Upload Trained Model to KaggleHub

---


In [ ]:
# print("--- Uploading Model 2 to KaggleHub ---")

# handle = "archie29/cnn_scratch2/pytorch/variation1" 
# local_model_path = "cnn_scratch2.pth"

# try:
#     kagglehub.model_upload(handle, 
#                            local_model_path, 
#                            version_notes="Second scratch CNN (v2) with BN, Dropout, and balanced loss.")
#     print(f"Successfully uploaded model to {handle}")
# except Exception as e:
#     print(f"Model upload failed: {e}")

[ScratchCNN2](https://www.kaggle.com/models/archie29/cnn_scratch2/pytorch/variation1)

In [ ]:
# print("--- Uploading Model 3 to KaggleHub ---")

# handle = "archie29/Pretrained_1/pytorch/variation1" 
# local_model_path = "Pretrained_1.pth"

# try:
#     kagglehub.model_upload(handle, 
#                            local_model_path, 
#                            version_notes="Model 3 -- v2: Trained with fixed LR (1e-4) and 2-phase training.")
#     print(f"Successfully uploaded model to {handle}")
# except Exception as e:
#     print(f"Model upload failed: {e}")


[Model3V1](https://www.kaggle.com/models/archie29/Pretrained_1/pytorch/variation1)


[Model3V2](https://www.kaggle.com/models/archie29/Pretrained_1/pytorch/variation1)

# 🔍3. Inference code
---


## 📥a. Load Uploaded Model from KaggleHub
---



In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_HANDLE = "archie29/Pretrained_1/pytorch/variation1" 
MODEL_FILE_NAME = "Pretrained_1.pth"

print(f"Loading model from {MODEL_HANDLE}...")
try:
    model_dir = kagglehub.model_download(MODEL_HANDLE)
    model_path = os.path.join(model_dir, MODEL_FILE_NAME)
    model = ResNet34MultiTask() 
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None 
    print("WARNING: Using a dummy model. Predictions will be random.")

Loading model from archie29/Pretrained_1/pytorch/variation1...


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 93.5MB/s]


Model loaded successfully.


In [5]:
#Test DataLoader 
test_img_dir = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test"
test_csv = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"

transform_Pretrained = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

test_dataset = FaceDataset(test_csv, test_img_dir, transform=transform_Pretrained, train=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)
print(f"Test DataLoader created with {len(test_dataset)} images.")

Test DataLoader created with 8677 images.


## 📈b. Test Prediction / Inference  

---



In [7]:
print("--- Starting Inference for Model 3 (ResNet34) ---")
all_ids = []
all_ages = []
all_genders = []

if model is not None:
    with torch.no_grad():
        for imgs, ids in test_loader:
            imgs = imgs.to(device)
            
            pred_age, pred_gender = model(imgs)
            
            ages_batch = pred_age.squeeze().cpu().numpy()
            genders_batch = pred_gender.argmax(dim=1).cpu().numpy()

            all_ids.extend(ids.numpy()) 
            all_ages.extend(ages_batch)
            all_genders.extend(genders_batch)
else:
    print("Generating random predictions as model failed to load.")
    all_ids = list(pd.read_csv(test_csv)['id'])
    all_ages = np.random.uniform(20, 50, len(all_ids))
    all_genders = np.random.randint(0, 2, len(all_ids))


print("Creating submission file...")
submission_df = pd.DataFrame({
    "id": all_ids,
    "age": all_ages,
    "gender": all_genders
})

# Post-processing
submission_df["age"] = submission_df["age"].clip(0) 
submission_df["age"] = np.round(submission_df["age"], 1) 
submission_df["gender"] = submission_df["gender"].astype(int) 

submission_df.to_csv("submission.csv", index=False)
print("submission.csv created successfully!")
print(submission_df.head())

--- Starting Inference for Model 3 (ResNet34) ---
Creating submission file...
submission.csv created successfully!
   id        age  gender
0   0  36.299999       1
1   1  31.900000       0
2   2  23.900000       1
3   3  33.200001       1
4   4  61.200001       1
